# Visualise the BBBP dataset using Uniform Manifold Approximation and Projection (UMAP)

**Content**:
- Create 2D visualization of Blood Brain Barrier Permeability (BBBP) dataset with UMAP
- Embed compounts of interest into the embedding

**The dataset**:  
The BBBP (Blood-Brain Barrier Penetration) dataset from [MoleculeNet](https://moleculenet.org/datasets-1) contains information on compounds and their ability to penetrate the blood-brain barrier, which is crucial for drug discovery and development.

**The method**:  
The UMAP method is an example of *unsupervised* machine learning that can embed high dimensional data in a lower dimensional space. Its advantages are that it preserves the local and global structure of the data.

**Methodology**:
- This notebook has gaps (marked with a `#TODO`) that you have to fill. 
- If all gaps are filled in correctly, the notebook cells can be run.
- A solution notebook with all gaps filled is provided in the GitHub repository.

This notebook has been created being inspired by or using content from [Source Notebook on Google Colab](https://colab.research.google.com/gist/ElanaPearl/444b3331f61485bbe8862db27cb2b968/mapping-chemical-space-with-umap.ipynb#scrollTo=dzmJAwfiAi6k)

## 1. Import libraries and format data
Here all necessary libraries are imported and the necessary data is fetched.

- `os`: Provides functions to interact with the operating system, such as checking file existence.
- `pandas`: Used for data manipulation and analysis.
- `numpy`: Provides support for numerical computations.
- `seaborn`: A visualization library based on matplotlib for statistical graphics.
- `matplotlib.pyplot`: A plotting library for creating static, animated, and interactive visualizations.
- `rdkit`: A toolkit for cheminformatics, used for handling chemical information.
- `sklearn`: A machine learning library for data analysis and modeling.
- `umap.umap_`: A library for dimensionality reduction using UMAP (Uniform Manifold Approximation and Projection).

In [ ]:
# install packages that are not available in google colab
%pip install rdkit

In [ ]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import rdkit
import sklearn
import umap.umap_ as umap

from rdkit.Chem import AllChem
from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser

In [ ]:
# Silence non-critical RDKit warnings to minimize unnecessary outputs
lg = rdkit.RDLogger.logger()
lg.setLevel(rdkit.RDLogger.CRITICAL)

We also define the following three helper functions that we need later in the notebook.

In [ ]:
def get_largest_fragment_from_smiles(s: str):
    """
    Get the largest fragment from a SMILES string.
    Args:
        s: SMILES string
    Returns:
        str: SMILES string of the largest fragment
    """
    mol = rdkit.Chem.MolFromSmiles(s)
    if mol:
        clean_mol = LargestFragmentChooser().choose(mol)
        return rdkit.Chem.MolToSmiles(clean_mol)
    return None

def compute_ecfp_descriptors(smiles_list):
    """ 
    Computes ecfp descriptors for a list of SMILES strings.
    Args:
        smiles_list: list of SMILES strings
    Returns: 
        np.array: ECFP descriptors
        list: list of indices of the SMILES strings that were successfully converted
    """

    keep_idx = []
    descriptors = []
    for i, smiles in enumerate(smiles_list):
        ecfp = _compute_single_ecfp_descriptor(smiles)
        if ecfp is not None:
            keep_idx.append(i)
            descriptors.append(ecfp)

    return np.vstack(descriptors), keep_idx

def _compute_single_ecfp_descriptor(smiles: str):
    """"
    Computes ECFP descriptors for a single SMILES string.
    Args:
        smiles: SMILES string
    Returns:
        np.array: ECFP descriptors
    """
    try:
        mol = rdkit.Chem.MolFromSmiles(smiles)
    except Exception as E:
        return None

    if mol:
        fp = rdkit.Chem.AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
        return np.array(fp)

    return None

Load BBBP dataset from the internet if neccesary (For example if this notebook is used in a Google colab, the data has to be retrieved and saved from the online GitHub repository).

In [ ]:
import urllib.request

if not os.path.exists('data/BBBP.csv'):
    os.makedirs('data', exist_ok=True)
    url = "https://github.com/moritzmarquardt/workshop_IBEC/raw/refs/heads/main/data/BBBP.csv"
    urllib.request.urlretrieve(url, "data/BBBP.csv")
    print("Data loaded from URL and saved locally")

bbbp = pd.read_csv("data/BBBP.csv")
bbbp.head()

### Format the BBBP dataset
- Change column names for easier interpretability
- remove extra fragments in the SMILES representation that is irrelevant for the brain-blood-barrier permeability (typically salts)


In [ ]:
# Clean up columnn names so they are easier to interpret
bbbp = bbbp[["smiles", "p_np", "name"]].reset_index(drop=True).rename({"p_np": "permeable"}, axis=1)

#reorder columns
bbbp = bbbp[["name", "smiles", "permeable"]]

# Remove extra fragments in SMILES 
bbbp["smiles"] = bbbp["smiles"].apply(get_largest_fragment_from_smiles).dropna()

pd.DataFrame(bbbp).head(10)

## 2. Transform SMILES (strings) to ECFP (binary) encoding

In this section, the Extended Connectivity Fingerprints (ECFPs) are computed for the SMILES strings in the dataset. ECFPs are a type of molecular descriptor that encodes the structure of a molecule into a fixed-length (2048 dimensions) binary vector. Read more about [ECFPs](https://pubs.acs.org/doi/10.1021/ci100050t).

In [ ]:
# Compute desrciptors and keep track of which failed to featurize
ecfp_descriptors, keep_idx = compute_ecfp_descriptors(bbbp["smiles"])

# Only keep those that sucessfully featurized
bbbp = bbbp.iloc[keep_idx]

pd.DataFrame(ecfp_descriptors).head()

## 3. Calculate UMAP embedding of ECFP encodings

**TODO**:
- Use the umap-learn library to define and fit a umap.UMAP object to the ecfp_descriptors

In [ ]:
#TODO define the umap_reducer object as a umap.UMAP object with the following parameters:
# metric = "jaccard" (because we use ecfp_descriptors which are binary)
# n_neighbors = 25 (effect: how much local information is preserved)
# n_components = 2 (effect: how many dimensions the data is reduced to)
# low_memory = False
# min_dist = 0.001 (effect: how much the points are spread out in the embedding space)
umap_reducer = 

#TODO use the fit_transform method of the umap_reducer object to transform the ecfp_descriptors and assign the result to the umap_embedding variable
umap_embedding = 

# Assign the UMAP coordinates to the bbbp dataframe
bbbp["UMAP_0"], bbbp["UMAP_1"] = umap_embedding[:,0], umap_embedding[:,1]

pd.DataFrame(bbbp).head()

#TODO (for fast people) Try different parameters for the UMAP object and see how the results change

## 4. Visualise UMAP

A scatter plot is created using the UMAP coordinates (`UMAP_0` and `UMAP_1`) from the `bbbp` dataframe. The compounds are color-coded based on their permeability (`permeable` column).

In [ ]:
palette = sns.color_palette(["hotpink", "dodgerblue"])
plt.figure(figsize=(5, 5))
sns.scatterplot(data=bbbp,
                x="UMAP_0",
                y="UMAP_1",
                hue="permeable",
                alpha=0.5,
                palette=palette)
plt.title(f"UMAP Embedding of BBBP Dataset", fontsize=10)
plt.show()

## 5. Project compounds of interest onto the existing UMAP

Here, we select a few compounds of interest and calculate their ECFP descriptors to embed them into the existing UMAP.

**TODO**:
1. Project (transform) the compund ECFP descriptors onto the previously calculated UMAP.

**Source of smiles descriptors**:
At [PubChem](https://pubchem.ncbi.nlm.nih.gov/) you can find smiles descriptors for chemical compounds.

In [ ]:
# embed a few compounds of interest into the umap
compounds_of_interest = pd.DataFrame()
compounds_of_interest["name"] = [
    "Caffeine (1)", 
    "Penicillin G (0)", 
    "Ethanol (1)",
    "Gold"
    ]
compounds_of_interest["smiles"] = [
    "CN1C=NC2=C1C(=O)N(C(=O)N2C)C", 
    "CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C", 
    "CCO",
    "[Au]"
    ]
# 0 for non-permeable, 1 for permeable, 2 for undefined
compounds_of_interest["permeable"] = [
    1, 
    0, 
    1,
    2
    ]
compounds_of_interest

In [ ]:
ecfp_compounds_of_interest, keep_idx = compute_ecfp_descriptors(compounds_of_interest["smiles"])

# Only keep those that sucessfully featurized
compounds_of_interest = compounds_of_interest.iloc[keep_idx]

pd.DataFrame(ecfp_compounds_of_interest)


In [ ]:
#TODO use the transform method of the umap_reducer object to transform the ecfp_compounds_of_interest
transformed_compounds_of_interest = 

compounds_of_interest["UMAP_0"], compounds_of_interest["UMAP_1"] = transformed_compounds_of_interest[:,0], transformed_compounds_of_interest[:,1]
compounds_of_interest

In [ ]:
plt.figure(figsize=(5, 5))
palette = sns.color_palette(["hotpink", "dodgerblue", "grey"])
sns.scatterplot(data=bbbp,
                x="UMAP_0",
                y="UMAP_1",
                hue="permeable",
                alpha=0.5,
                palette=palette)
colors = ["black", "red", "blue", "green"]
for i, row in compounds_of_interest.iterrows():
    plt.scatter(row["UMAP_0"], row["UMAP_1"], s=100, edgecolor='black', facecolor=palette[row["permeable"]])
    plt.text(row["UMAP_0"] + 0.7, row["UMAP_1"], row["name"], fontsize=10, va='center', bbox=dict(facecolor='white', alpha=0.8))
plt.legend()
plt.show()


#TODO (for fast people): Go to PubChem and find a molecule of interest, compute its ECFP descriptors and embed it into the UMAP plot.

Takeaway from this last section:  
We can project own compounds of interest onto the existing UMAP embedding and by this see, if the compounds of interest are close in terms of their structure to other compounds in the BBBP dataset.